
# Transformación de Datos: Capa Silver en Azure Databricks

Este notebook lee datos desde la capa Bronze (Delta Lake gestionada) y aplica transformaciones y limpieza antes de escribir en la **capa Silver** utilizando `MERGE INTO` para control de cambios.


In [0]:
from pyspark.sql.functions import upper
from delta.tables import DeltaTable

In [0]:
def leer_desde_bronze(nombre_tabla: str, catalog_name: str = "desarrollo", db_bronze: str = "bronze_clinica"):
    full = f"{catalog_name}.{db_bronze}.{nombre_tabla}"
    if not spark.catalog.tableExists(full):
        raise ValueError(f"La tabla {full} no existe en el metastore.")
    return spark.table(full)  # o spark.read.table(full)


In [0]:
from pyspark.sql.functions import col, to_timestamp, when

# Leer desde bronze
df_Camas = leer_desde_bronze("t_camas", db_bronze="bronze_clinica")

# Paso 1: Manejo de valores nulos
df_Camas = df_Camas.fillna({
    "numero_cama": 0,  # Reemplaza nulos en numero_cama  con 0
    "numero_habitacion": 0,     # Reemplaza nulos en numero_habitacion  con 0
    "fecalta": "1970-01-01 00:00:00"  # Reemplaza fechas nulas con una fecha por defecto
})

# Paso 2: Eliminación de duplicados (basado en todas las columnas)
df_Camas = df_Camas.dropDuplicates()

# Mostrar la estructura del DataFrame después de la limpieza
df_Camas.printSchema()

# Contar filas antes y después de la limpieza para validar cambios
print(f"Total de filas después de la limpieza: {df_Camas.count()}")



In [0]:
# Leer desde bronze
df_Habitaciones = leer_desde_bronze("t_habitaciones", db_bronze="bronze_clinica")

# Paso 1: Manejo de valores nulos
df_Habitaciones = df_Habitaciones.fillna({
    "numero_habitacion": 0, "piso": 0,  # Reemplaza nulos en numero_cama  con 0
    "piso": 0,  # Reemplaza nulos en numero_cama  con 0
    "fecalta": "1970-01-01 00:00:00"  # Reemplaza fechas nulas con una fecha por defecto
})

# Paso 2: Eliminación de duplicados (basado en todas las columnas)
df_Habitaciones = df_Habitaciones.dropDuplicates()

# Mostrar la estructura del DataFrame después de la limpieza
df_Habitaciones.printSchema()

# Contar filas antes y después de la limpieza para validar cambios
print(f"Total de filas después de la limpieza: {df_Habitaciones.count()}")

In [0]:
# Leer desde bronze
df_Pacientes = leer_desde_bronze("t_pacientes", db_bronze="bronze_clinica")

# Paso 1: Manejo de valores nulos
df_Pacientes = df_Pacientes.fillna({
    "id_paciente": 0,  "genero": 0,  # Reemplaza nulos en numero_cama  con 0
    "fecalta": "1970-01-01 00:00:00"  # Reemplaza fechas nulas con una fecha por defecto
})

# Paso 2: Eliminación de duplicados (basado en todas las columnas)
df_Pacientes = df_Pacientes.dropDuplicates()

# Mostrar la estructura del DataFrame después de la limpieza
df_Pacientes.printSchema()

# Contar filas antes y después de la limpieza para validar cambios
print(f"Total de filas después de la limpieza: {df_Pacientes.count()}")

In [0]:
# Leer desde bronze
df_Medicos = leer_desde_bronze("t_medicos", db_bronze="bronze_clinica")

# Paso 1: Manejo de valores nulos
df_Medicos = df_Medicos.fillna({
    "id_medico": 0, "estado": 0,  # Reemplaza nulos en numero_cama  con 0
    "fecalta": "1970-01-01 00:00:00"  # Reemplaza fechas nulas con una fecha por defecto
})

# Paso 2: Eliminación de duplicados (basado en todas las columnas)
df_Medicos = df_Medicos.dropDuplicates()

# Mostrar la estructura del DataFrame después de la limpieza
df_Medicos.printSchema()

# Contar filas antes y después de la limpieza para validar cambios
print(f"Total de filas después de la limpieza: {df_Medicos.count()}")

In [0]:
# Leer desde bronze
df_Internamientos = leer_desde_bronze("t_internamientos", db_bronze="bronze_clinica")

# Paso 1: Manejo de valores nulos
df_Internamientos = df_Internamientos.fillna({
    "id_internamiento": 0, "id_paciente": 0,  # Reemplaza nulos en numero_cama  con 0
    "id_medico": 0, "numero_cama": 0,  # Reemplaza nulos en numero_cama  con 0
    "fecalta": "1970-01-01 00:00:00"  # Reemplaza fechas nulas con una fecha por defecto
})

# Paso 2: Eliminación de duplicados (basado en todas las columnas)
df_Internamientos = df_Internamientos.dropDuplicates()

# Mostrar la estructura del DataFrame después de la limpieza
df_Internamientos.printSchema()

# Contar filas antes y después de la limpieza para validar cambios
print(f"Total de filas después de la limpieza: {df_Internamientos.count()}")

In [0]:
# Leer desde bronze
df_VisitasMedicas = leer_desde_bronze("t_visitas_medicas", db_bronze="bronze_clinica")

# Paso 1: Manejo de valores nulos
df_VisitasMedicas = df_VisitasMedicas.fillna({
    "id_visita": 0, "id_internamiento": 0,  # Reemplaza nulos en numero_cama  con 0
    "id_medico": 0,  # Reemplaza nulos en numero_cama  con 0
    "fecalta": "1970-01-01 00:00:00"  # Reemplaza fechas nulas con una fecha por defecto
})

# Paso 2: Eliminación de duplicados (basado en todas las columnas)
df_VisitasMedicas = df_VisitasMedicas.dropDuplicates()

# Mostrar la estructura del DataFrame después de la limpieza
df_VisitasMedicas.printSchema()

# Contar filas antes y después de la limpieza para validar cambios
print(f"Total de filas después de la limpieza: {df_VisitasMedicas.count()}")

In [0]:
%sql
USE CATALOG desarrollo;

SHOW DATABASES;

In [0]:
from typing import List, Optional
from delta.tables import DeltaTable

def crear_tabla_delta_merge_managed(
    nombre_df: str,
    nombre_tabla: str,
    llave_origen: List[str],
    llave_destino: List[str],
    db_name: str = "default",
    catalog_name: str = "desarrollo",
    partition_cols: Optional[List[str]] = None,
    auto_merge_schema: bool = True
) -> None:
    """
    Crea si no existe una tabla Delta GESTIONADA en la base (que ya debe tener LOCATION en tu mount)
    y realiza MERGE. No usa LOCATION explícito.
    """

    # Validaciones
    df = globals()[nombre_df]

    if len(llave_origen) != len(llave_destino):
        print("❌ Error: La cantidad de columnas en 'llave_origen' y 'llave_destino' no coinciden.")
        return

    if partition_cols:
        faltantes = [c for c in partition_cols if c not in df.columns]
        if faltantes:
            print(f"❌ Error: Columnas de partición no existen en el DataFrame: {faltantes}")
            return

    if auto_merge_schema:
        spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

    # Armar nombre completo
    full_name = f"{catalog_name}.{db_name}.{nombre_tabla}"

    # ✅ FIX: usar el overload moderno (una sola cadena)
    exists = spark.catalog.tableExists(full_name)

    if not exists:
        # Crear como TABLA GESTIONADA en el LOCATION de la DB (sin LOCATION explícito)
        writer = df.write.format("delta").mode("overwrite")
        if partition_cols:
            # ✅ FIX: varargs
            writer = writer.partitionBy(*partition_cols)
        writer.saveAsTable(full_name)
        print(f"✅ Tabla gestionada creada: {full_name} (bajo LOCATION de la base '{db_name}')")
        return

    # Si existe, MERGE
    try:
        delta_tbl = DeltaTable.forName(spark, full_name)
    except Exception as e:
        raise RuntimeError(f"❌ La tabla {full_name} no es Delta o no es accesible como Delta: {e}")

    merge_condition = " AND ".join(
        [f"tgt.`{llave_destino[i]}` = src.`{llave_origen[i]}`" for i in range(len(llave_origen))]
    )
    set_expr  = {c: f"src.`{c}`" for c in df.columns}
    vals_expr = {c: f"src.`{c}`" for c in df.columns}

    print(f"🔄 Ejecutando MERGE INTO {full_name} ...")
    (delta_tbl.alias("tgt")
             .merge(df.alias("src"), merge_condition)
             .whenMatchedUpdate(set=set_expr)
             .whenNotMatchedInsert(values=vals_expr)
             .execute())
    print(f"✅ MERGE completado para {full_name}")


In [0]:
# Ejecutar la función para crear la tabla y hacer MERGE usando diferentes llaves
crear_tabla_delta_merge_managed(
    nombre_df="df_Camas",
    nombre_tabla="md_camas",
    llave_origen=["numero_cama"],
    llave_destino=["numero_cama"],
    db_name="silver_clinica",
    partition_cols=["fecalta"]  # opcional; si no quieres partición, quítalo
)

crear_tabla_delta_merge_managed(
    nombre_df="df_Habitaciones",
    nombre_tabla="md_habitaciones",
    llave_origen=["numero_habitacion"],
    llave_destino=["numero_habitacion"],
    db_name="silver_clinica",
    partition_cols=["fecalta"]  # opcional; si no quieres partición, quítalo
)

crear_tabla_delta_merge_managed(
    nombre_df="df_Pacientes",
    nombre_tabla="md_pacientes",
    llave_origen=["id_paciente"],
    llave_destino=["id_paciente"],
    db_name="silver_clinica",
    partition_cols=["fecalta"]  # opcional; si no quieres partición, quítalo
)

crear_tabla_delta_merge_managed(
    nombre_df="df_Medicos",
    nombre_tabla="md_medicos",
    llave_origen=["id_medico"],
    llave_destino=["id_medico"],
    db_name="silver_clinica",
    partition_cols=["fecalta"]  # opcional; si no quieres partición, quítalo
)

crear_tabla_delta_merge_managed(
    nombre_df="df_Internamientos",
    nombre_tabla="md_internamientos",
    llave_origen=["id_internamiento"],
    llave_destino=["id_internamiento"],
    db_name="silver_clinica",
    partition_cols=["fecalta"]  # opcional; si no quieres partición, quítalo
)

crear_tabla_delta_merge_managed(
    nombre_df="df_VisitasMedicas",
    nombre_tabla="md_visitas_medicas",
    llave_origen=["id_visita"],
    llave_destino=["id_visita"],
    db_name="silver_clinica",
    partition_cols=["fecalta"]  # opcional; si no quieres partición, quítalo
)
